In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Dataset processing

In [ ]:
import numpy as np
from astropy.io import fits
import configparser

In [ ]:
config_object = configparser.ConfigParser()
with open("process_config.ini","r") as file_object:
    config_object.read_file(file_object)
    path = config_object.get("path","path")
    save = config_object.get("path","save")
    general = config_object.get("path","general")
    nbins_freq = int(config_object.get("params","nbins_freq"))
    freqs = np.linspace(float(config_object.get("params","min_freq")),
                        float(config_object.get("params","max_freq")), nbins_freq+1)

In [ ]:
file = fits.open(general)
general_header = file['PRIMARY'].header
general_shape = file['PRIMARY'].data.shape
# read data from a single cube
if path == 'general':
    data = file['PRIMARY'].data
file.close()
# read data from various 2D images (for polyfit)
if path != 'general':
    data = np.zeros(general_shape)
    for i in range(general_shape[0]):
        with fits.open(path+'_%s.fits'%i) as file:
            data[i,:,:] = file['PRIMARY'].data
        if i%100 == 0:
            print(i)

The next cell calculates the number of frequency channels in each bin, and in case the cube cannot be divided in equal bins, saves the remainder to add it in the last bin.

In [ ]:
remainder = general_shape[0]%nbins_freq
width = int((general_shape[0]-reminder)/nbins_freq)

In [ ]:
for i in range(nbins_freq):
    header = general_header
    
    header['SIMPLE'] = ('T', 'file does conform to FITS standard')            
    header['BITPIX'] = (-32, 'number of bits per data pixel')
    header['CRVAL3'] = freqs[i]*1e6
    if i != nbins_freq-1:
        header['NAXIS3'] = (width, 'length of data axis 3')
        hdu = fits.PrimaryHDU(data=data[i*(width+1)-i:(width+1)*(i+1)-(i+1)].astype(np.float32), header=header)
    else:
        header['NAXIS3'] = (width+reminder, 'length of data axis 3')
        hdu = fits.PrimaryHDU(data=data[i*(width+1)-i:(width+1+reminder)*(i+1)-(i+1)].astype(np.float32), header=header)
    hdu.writeto(save+'_%sMHz-%sMHz.fits'%(freqs[i],freqs[i+1]))
    print(i)